# Autism Screening on Adults using Linear Regression

Esse notebook será usado para treinar as habilidade de **Linear Regression** que eu obtive no livro **Introduction To Statistical Learning**

# 0.1 Libs

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm

# 0.2 Data

In [29]:
cwd = os.getcwd()
df = pd.read_csv(cwd + "\\..\\data\\autism_screening.csv")

df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   A1_Score         704 non-null    int64  
 1   A2_Score         704 non-null    int64  
 2   A3_Score         704 non-null    int64  
 3   A4_Score         704 non-null    int64  
 4   A5_Score         704 non-null    int64  
 5   A6_Score         704 non-null    int64  
 6   A7_Score         704 non-null    int64  
 7   A8_Score         704 non-null    int64  
 8   A9_Score         704 non-null    int64  
 9   A10_Score        704 non-null    int64  
 10  age              702 non-null    float64
 11  gender           704 non-null    object 
 12  ethnicity        704 non-null    object 
 13  jundice          704 non-null    object 
 14  austim           704 non-null    object 
 15  contry_of_res    704 non-null    object 
 16  used_app_before  704 non-null    object 
 17  result          

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,United States,no,6.0,18 and more,Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5.0,18 and more,Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8.0,18 and more,Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,United States,no,6.0,18 and more,Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,f,?,no,no,Egypt,no,2.0,18 and more,?,NO


# 0.3 Understanding the Data

### O que são esses scores de A1 até A10?
Aparentemente Scores A1 até A10 são perguntas contidas nesse link: https://docs.autismresearchcentre.com/tests/AQ10.pdf

### O que significa o 0 e o 1 nas colunas?
Pelo que vi das perguntas, eles usam mais de 2 respostas, sendo elas Concordo bastante, Concordo pouco, etc. Já no caso desse dataframe, parece que só vai envolver **1 = Sim** e **2 = Não**

# 0.4 Cleaning Data

In [30]:
# Tratei os missing values da coluna Relation criando uma nova categoria
df['relation'] = df['relation'].replace('?', 'Unknown')

# Tratei os missing values da coluna Ethnicity criando uma nova categoria
df['ethnicity'] = df['ethnicity'].replace('?', 'Unknown')

# Autism estava escrito errado
df = df.rename(columns={'austim': 'autism'})

# 0.5 Creating Dummy Variables

In [31]:
# remover age_desc
# gender, ethnicity, jundice, country_of_res, used_app_before, relation, Class/ASD
# Disclaimer: À partir do momento em que eu comecei a criar meu modelo, me deparei com diversos dados qualitativos, o que dificulta o uso de uma Regressão linear

#Used_App_Before
df['used_app_before'] = pd.get_dummies(df['used_app_before'], prefix=['used_app_before'], drop_first=True)
df['used_app_before'].replace(0, -1, inplace=True)
df.info()

#Autism
df['autism'] = pd.get_dummies(df['autism'], prefix=['autism'], drop_first=True)
df['autism'].replace(0, -1, inplace=True)

#Gender M = 1 | F = -1
df['gender'] = pd.get_dummies(df['gender'], prefix=['gender'], drop_first=True)
df['gender'].replace(0, -1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   A1_Score         704 non-null    int64  
 1   A2_Score         704 non-null    int64  
 2   A3_Score         704 non-null    int64  
 3   A4_Score         704 non-null    int64  
 4   A5_Score         704 non-null    int64  
 5   A6_Score         704 non-null    int64  
 6   A7_Score         704 non-null    int64  
 7   A8_Score         704 non-null    int64  
 8   A9_Score         704 non-null    int64  
 9   A10_Score        704 non-null    int64  
 10  age              702 non-null    float64
 11  gender           704 non-null    object 
 12  ethnicity        704 non-null    object 
 13  jundice          704 non-null    object 
 14  autism           704 non-null    object 
 15  contry_of_res    704 non-null    object 
 16  used_app_before  704 non-null    int16  
 17  result          

# 0.6 Modelling

In [38]:
y_train = df['autism']
x_train = df['result']

x_train = sm.add_constant(x_train)
model = sm.OLS(y_train, x_train)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 autism   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     26.88
Date:                Thu, 25 May 2023   Prob (F-statistic):           2.84e-07
Time:                        10:01:18   Log-Likelihood:                -704.80
No. Observations:                 704   AIC:                             1414.
Df Residuals:                     702   BIC:                             1423.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9928      0.054    -18.226      0.0

# 0.7 Conclusion

Bom, após alguns testes e estudos, cheguei a conclusão de que esse dataframe não é muito bom para ser usado em uma Linear Regression. Desde o começo, ao ver que existiam vários dados qualitativos
já entendi que esse dataframe não seria o mais correto. Ao fim, aprendi que quando temos um Y(Target Value) binário, ou seja, com apenas tipos de dados, nesse caso Sim e Não, a regressão linear acaba não sendo muito útil.

Não vejo esse mini-projeto como algo que tenha sido inútil, já que dentro dele descobri diversas coisas e aprendi várias outras.